# Continuations

### Examples

https://docs.google.com/presentation/d/1OlAu7pjBfKogQg6Z5Ykkid9FfWVH-yrSbPUCHMI-3HM/edit?usp=sharing

## Exercise
Given the following functions, change them to use continuations:

In [8]:
def doThing1(): Int = 5 + 3
def doThing2(a: Int, b: Int): Int = a + b
def doThing3(f: Int => String): Int = {
    println(f(5))
    5
}

defined function doThing1
defined function doThing2
defined function doThing3

In [7]:
// YOUR CODE HERE
def doThing1_cont[T](k : Int=>T): T = k(5+3)

doThing1_cont(a => a.toString + " Hello World!")

def doThing2_cont[T](a: Int, b:Int, k : Int => T):T = k(a + b)

doThing2_cont(3,4,a => a.toString + " Hello World!" )

def doThing3_cont[T](f: Int => String, k: Int=>T): T = {
    println(f(5))
    k(5)
}

defined function doThing1_cont
res6_1: String = "8 Hello World!"
defined function doThing2_cont
res6_3: String = "7 Hello World!"
defined function doThing3_cont

## Exercise
Update the following expression to use the new continuation functions:

In [9]:
val x = doThing1()
val y = doThing2(x, 3)
val z = doThing3(n => "Log: " + n.toString)

Log: 5


x: Int = 8
y: Int = 11
z: Int = 5

In [10]:
// YOUR CODE HERE

assert(z == z2)

Log: 5


z2: Int = 5

### Exercise: Backtracking
As with any good programing tool, we can use continuations to solve problems without putting them everywhere in our code. In this example, you will write a search function that looks for a value in a binary tree, **not** a B**S**T. Take advantage of continuations to remember where to look if something isn't found while keeping the function tail recursive.

In [11]:
sealed trait Tree
case object Empty extends Tree
case class Node(l: Tree, d: Int, r: Tree) extends Tree

def search[T](t: Tree, i: Int, fail_continuation: () => Boolean): Boolean =
    /*
        t match {
            case Empty => false
            case Node(l, j, r) if j == i => true
            case Node(l, j, r) => {
                val v1 = search(l, j)
                val v2 = search(r, j)
                v1 && v2
            }
        }
    */
    // YOUR CODE HERE
    t match {
        case Empty => fail_continuation()
        case Node(_,j,_) if i == j => true
        case Node(l, j, r) =>
            search(l, i, () => {
                search(r, i, fail_continuation)
            })
    }

defined trait Tree
defined object Empty
defined class Node
defined function search

In [12]:
val t = Node(Empty, 10, Node(Empty, 6, Empty))
assert(search(t, 10, () => false))
assert(search(t, 6, () => false))
assert(!search(t, 0, () => false))

t: Node = Node(Empty, 10, Node(Empty, 6, Empty))

### Exercise: Eval (again)

This is similar to the example in class: implement eval for the small language given below, ensuring that every function call is a tail call.

In [13]:
sealed trait Expr
case class BoolLiteral(b: Boolean) extends Expr
case class And(left: Expr, right: Expr) extends Expr
case class If(test: Expr, then: Expr, otherwise: Expr) extends Expr

def eval(e: Expr, continuation: Boolean => Boolean): Boolean =
    // YOUR CODE HERE
    e match{
        case BoolLiteral(b) => continuation(b)
        case And(l,r) => 
            eval(l, left_val => 
                eval(r, right_val =>
                    continuation(left_val && right_val)))
        case If(test, then, otherwise) => 
            eval(test, test_val =>
                if(test_val) eval(then, b=> continuation(b))
                 else eval(otherwise, continuation)
                )
    }

defined trait Expr
defined class BoolLiteral
defined class And
defined class If
defined function eval

In [16]:
val e = If(
    And(BoolLiteral(true), BoolLiteral(true)),
    BoolLiteral(false),
    BoolLiteral(true)
)
assert(!eval(e, x => x))

eval(And(BoolLiteral(true), BoolLiteral(false)), x=>x)

e: If = If(
  And(BoolLiteral(true), BoolLiteral(true)),
  BoolLiteral(false),
  BoolLiteral(true)
)
res15_2: Boolean = false

## Exercise: Async Continuations (Callbacks)
Use the `http_get` function to request the given url and provide a continuation that will append `" lie."` to the result

In [17]:
import $ivy.`com.lihaoyi::requests:0.1.4`
import scala.concurrent.{ Future, Await }
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global

def http_get(url: String, k: String => String): Future[String] =
    Future(requests.get(url)).map(_.text).map(k)

import $ivy.$                            

import scala.concurrent.{ Future, Await }

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext.Implicits.global


defined function http_get

In [21]:
val shakira_url = "http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6"

var done: Boolean = false
val res = http_get(shakira_url, result => {done = true; result + " lie."})
println("request made")
println(s"is done: $done")

request made
is done: false


shakira_url: String = "http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6"
done: Boolean = true
res: Future[String] = Success("Hips don't lie.")

In [19]:
// Run this cell if you're using the older scala kernel (from VM)
// which doesn't update the future
Await.result(res, 5000 millis)

res18: String = "Hips don't lie."

```c#
await
```